In [1]:
import os
%pwd

'c:\\Users\\Sasu4\\SHIP_Classification_using_Resnet\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\Sasu4\\SHIP_Classification_using_Resnet'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    num_epochs: int
    batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    learning_rate : float
    class_names  : list
    
    


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    epoch:int
    
    
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    freeze_all: bool
    freeze_till: int
    
    

In [4]:
class PrepareBaseModel:
  def __init__(self, config):
        self.config = config
        self.model=None
        self.optimizer = None  # Initialize as needed
        self.loss_fn = None  # Initialize as needed

In [5]:
from Ship_Classifier.constants import *
from Ship_Classifier.utils.common import read_yaml,create_directories
import torch
import torch.nn as nn
from torchvision import models,transforms,datasets
import torch.optim as optim
from PIL import Image
from torch.utils.data import DataLoader

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params
        
        #create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            freeze_all=config.get("freeze_all", True),  # Default to True if not specified
            freeze_till=config.get("freeze_till", None)
        )

        return prepare_base_model_config

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
            epoch=self.params.EPOCHS
        )

        return prepare_callback_config
    
    
    def get_training_config(self) -> TrainingConfig:
       training = self.config.training
       prepare_base_model = self.config.prepare_base_model
       params = self.params
       
       # Update the path to match the directory structure shown in the screenshot
      # training_data = os.path.join(self.config.data_ingestion.unzip_dir, "extracted_data", "Images")
       create_directories([
         Path(training.root_dir)
    ])

       return TrainingConfig(
        root_dir=Path(training.root_dir),
        trained_model_path=Path(training.trained_model_path),  # This now points to the correct image folder structure
        updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
        training_data = Path(os.path.join(self.config.data_ingestion.unzip_dir)),
        batch_size=params.BATCH_SIZE,
        num_epochs=params.epochs,
        learning_rate=params.LEARNING_RATE,
        params_is_augmentation=params.AUGMENTATION,
        params_image_size=params.IMAGE_SIZE,
        class_names=self.config.CLASS_NAMES 
        #base_model_dir=prepare_base_model.model_dir,
        #base_model_name=prepare_base_model.model_name,
        # Call this function in your training code

    )



In [7]:
import time
import os
import urllib.request as request
from zipfile import ZipFile
from torch.utils.tensorboard import SummaryWriter

In [8]:
 
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig,model, optimizer, loss):
        self.config = config
        self.model = model
        self.optimizer = optimizer
        self.loss = loss
   
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return torch.utils.tensorboard.SummaryWriter(log_dir=tb_running_log_dir)  # Adjust as necessary to return the TensorBoard logger

    @property
    def _create_ckpt_callbacks(self):
       checkpoint_dir = self.config.checkpoint_model_filepath
       checkpoint = torch.save({
            'epoch': self.config.epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'loss': self.loss
       }, checkpoint_dir)
       return checkpoint
    
    def get_tb_ckpt_callbacks(self):
        tb_callback = self._create_tb_callbacks
        ckpt_callback = self._create_ckpt_callbacks
        return tb_callback, ckpt_callback

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def get_base_model(self):
        # Load a pre-trained ResNet-18 model
        self.model = models.resnet18(pretrained=True)
        
        # Modify the final fully connected layer to match the number of classes
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, len(self.config.class_names))
        
        # Load the updated base model if it exists
        if self.config.updated_base_model_path.exists():
            self.model.load_state_dict(torch.load(self.config.updated_base_model_path, map_location=self.device))
        
        self.model = self.model.to(self.device)

    def train_valid_loader(self):
        # Define transformations for the training and validation datasets
        train_transform = transforms.Compose([
            transforms.RandomResizedCrop(self.config.params_image_size[0]),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        valid_transform = transforms.Compose([
            transforms.Resize(self.config.params_image_size[0]),
            transforms.CenterCrop(self.config.params_image_size[0]),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        # Datasets and data loaders
        train_dataset = datasets.ImageFolder(self.config.training_data, transform=train_transform)
        valid_dataset = datasets.ImageFolder(self.config.training_data, transform=valid_transform)

        self.train_loader = DataLoader(train_dataset, batch_size=self.config.batch_size, shuffle=True)
        self.valid_loader = DataLoader(valid_dataset, batch_size=self.config.batch_size, shuffle=False)
    
    def display_image_counts(self,path: Path):
    # List of all class folders
     classes = [entry for entry in path.iterdir() if entry.is_dir()]

    # Iterate through each class folder and count the images
     for cls in classes:
        num_images = len(list(cls.glob('*.jpg'))) + len(list(cls.glob('*.png')))  # Update extensions if needed
        print(f"Class: {cls.name}, Number of images: {num_images}")

    def save_model(self, path: Path):
        torch.save(self.model.state_dict(), path)

    def train(self, callback_list: list = None):
        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.config.learning_rate)

        for epoch in range(self.config.num_epochs):
            # Training phase
            self.model.train()
            running_loss = 0.0
            for inputs, labels in self.train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                 

            epoch_loss = running_loss / len(self.train_loader.dataset)
           

            # Validation phase
            self.model.eval()
            val_loss = 0.0
            corrects = 0
            with torch.no_grad():
                for inputs, labels in self.valid_loader:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    outputs = self.model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * inputs.size(0)

                    pred = torch.argmax(outputs, 1)
                    corrects += torch.sum(pred == labels.data)
                    

            val_loss = val_loss / len(self.valid_loader.dataset)
            val_acc = corrects.double() / len(self.valid_loader.dataset)

            print(f'Epoch {epoch}/{self.config.num_epochs - 1}, '
                  f'Train Loss: {epoch_loss:.4f}, '
                  f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

            # You can include the callback logic here if you need to apply any callbacks

        self.save_model(self.config.trained_model_path)
        writer = SummaryWriter(log_dir='artifacts/prepare_callbacks/tensorboard_log_dir')

In [10]:
from Ship_Classifier.components.prepare_base_model import PrepareBaseModel

from Ship_Classifier.components.prepare_base_model import PrepareBaseModel
try:
    config = ConfigurationManager()

    #Prepare the base model configuration
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    model = prepare_base_model.get_base_model()
    full_model,optimizer,loss_fn,predict,evaluate_model=prepare_base_model.update_base_model()
    
    
    print(f"Model: {full_model}")
    print(f"Optimizer: {optimizer}")
    print(f"Loss: {optimizer}")

    # Prepare the callback configuration
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(
        config=prepare_callbacks_config,
        model=full_model,
        optimizer=optimizer,
        loss=loss_fn
    )
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()


    # Prepare training configurations
    training_config = config.get_training_config()
    
    # Initialize the training class with the training configuration
    training = Training(config=training_config)
    
    # Load the base model (ResNet-18)
    training.get_base_model()
    
    # Prepare data loaders for training and validation
    training.train_valid_loader()
    
    images_path = Path("artifacts/data_ingestion/extracted_data")
    training.display_image_counts(images_path)
    
    
    # Train the model with the specified callbacks
    training.train(callback_list=callback_list)
    
    
except Exception as e:
    raise Exception(f"An error occurred during the training process: {str(e)}") from e

[2024-08-28 10:10:19,143: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-28 10:10:19,160: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-28 10:10:19,162: INFO: common: created directory at: artifacts]
Base model loaded with 5 classes.
Model prepared with classes=5, freeze_all=True, freeze_till=None, learning_rate=0.01
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pa

c:\Users\Sasu4\miniconda3\envs\SHIP\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Sasu4\miniconda3\envs\SHIP\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Sasu4\AppData\Local\Temp\ipykernel_24416\2479646957.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/p

Class: Images, Number of images: 0


Exception: An error occurred during the training process: [Errno 2] No such file or directory: 'artifacts\\data_ingestion\\extracted_data\\Images\\Cruise\\2807234.jpg'